In [113]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import time
from requests_html import HTMLSession # https://github.com/psf/requests-html

On cherche par rapport au nom de famille, les noms de famille de naissance sont utilisés. Il faut donc regarder le lieu de naissance des enfants pour savoir si la famille à bouger (suivie le/la partenaire).

Il faudrait arriver à trouver l'ancêtre le plus loin pour un nom de famille, puis analyser sa descendance :
Pour chaque enfant : 
    - Si c'est un garcon ou une fille on récupère les informations nescessaires + récupération des informations de l'épouse/époux
    - Si c'est un garcon on analysera la descendance


Informations à récupérer pour chaque personne :
 - Genre
 - Nom
 - Prénom
 - Date naissance
 - Lieu de naissance
 - Date mort
 - Lieu de mort


Visualisation (folium):
 - Sur une carte qui s'actualise tous les 10 ans, on affiche les naissances et les morts aux endroits correspondants

Avec le lieu de naissance de toute la famille, on peut voir comment ils se sont déplacé au fil des ans.

Améliorations :

Utilisation du lieu de mort, du lieu de certains événements, des informations des époux/épouses




In [114]:
# set up 
# !pip install selenium
# !pip install bs4
# !pip install webdriver_manager
# !pip install requests


In [115]:
# initialisation

# vidéo tuto : https://www.youtube.com/watch?v=3fcKKZMFbyA 


data_test = [{"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"},{"nom":"MARTIN", "prénom":"Jacques", "genre": "M", "ddn": 1822, "ldn":"Lyon", "ddm": 1898, "ldm":"Paris"}]

# print(data_test)
data_test = sorted(data_test, key=lambda d: d['ddn']) 
# print(data_test)


In [116]:
# Futur fonction à utiliser (pour saisie du nom prenom et création de l'url correspondante)
def traitementURL():
    nom, prenom=getNomPrenom()
    url = "https://www.geneanet.org/fonds/individus/?go=1&nom=" + nom + "&prenom=" + prenom + "&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10" 
    return url 

def getNomPrenom():
    nom = input("Saisir nom")
    prenom = input("Saisir prénom")
    return nom, prenom


In [117]:
# Url de recherche de test
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&prenom=fran%C3%A7ois&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10"

In [118]:
# Récupération du code html
def getSoup(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

In [119]:
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&page=1&prenom=fran%C3%A7ois&prenom_operateur=or&size=10&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint="

def getAllArbreUrl(url):
    # Récupération du code html
    soup = getSoup(url)

    # Récupération de la div contenant toutes les urls en lien avec la recherche
    resOnly = soup.find("div", {"id":"table-resultats"})
    # print(resOnly)

    allArbreUrl = []
    # Récupération des urls
    for a in resOnly.find_all('a'):
        try:
            if (re.search("^arbres_utilisateur", a["data-id-es"])):  #^arbres_utilisateur
                allArbreUrl.append(a["href"])
        except:
            ""
    print(allArbreUrl) # Comporte l'url des pages utilisateurs si se sont des arbres utilisateurs

# getAllArbreUrl(url)


In [120]:
session = HTMLSession()

In [121]:
# Récupération des ascendants (remonter facilement à l'ancetre)
def getUrlDerAscendant(url):
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')
    # Récupération du dernier ascendants
    # print(soup)
    urlDerAscendant = url
    try: 
        arbreGene =  soup.find("td", {"id":"ancestors"})
        derAscendant = arbreGene.find("a")["href"]
        urlDerAscendant = "https://gw.geneanet.org/" + derAscendant
        urlDerAscendant = getUrlDerAscendant(urlDerAscendant)
        
    except:
        print("Pas d'autre ascendent")

    return urlDerAscendant

In [122]:
def getEpouxEnfant(data, person_html):
    # Récupération époux/épouse/enfants
    print("debut epouxenfant")
    person_union = person_html.find("ul", {"class":"fiche_union"})
    print("personunion")
    person_epoux = person_union.findChildren("li" , recursive=False)
    print("personepoux")
    urlEpoux = []
    urlChildren = []
    
    try:
        for epoux in person_epoux:
            urlEpoux.append("https://gw.geneanet.org/" + epoux.find("a")["href"])
            children = epoux.find_all("li")
            print(children)
            try:
                for child in children:
                    urlChildren.append("https://gw.geneanet.org/" + child.find("a")["href"])
            except:
                print("Err Children")
    except:
        print("Err Epoux")
    
    for url in urlEpoux:
        data = getDataUrl(data, url)
    for url in urlChildren:
        data = getDataUrl(data, url)
    return data

In [123]:
t = []
t2 = [{"z":5, 4:5},{"z":5, 4:5}]
t3 = [{"rzfe":"aaaaa"}]
t += t2
t += t3
print(t)

[{'z': 5, 4: 5}, {'z': 5, 4: 5}, {'rzfe': 'aaaaa'}]


In [124]:
def getDataUrl(data, url):
    personne = {}
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')

    
    # Récupération de l'html de l'ensemble des données d'une personne
    person_html = soup.find("div", {"id":"perso"})

    # Récupération des données concernant la personne (genre, nom, prenom)
    person_title = person_html.find("div", {"id":"person-title"})
    person_title = person_title.find("h1")

    personne["genre"] = person_title.find("img")["title"]
    
    nomprenom = person_title.find_all("a")
    personne["nom"] = nomprenom[0].text
    personne["prenom"] = nomprenom[1].text

    # Récupération des données concernant la personne (ddm, ldm, ddm, ldm)
    person_data_perso = person_html.find("ul")

    ldn = ""
    ldm = ""

    for li in person_data_perso.findAll("li"):
        li = li.text

        try:
            personne["ddn"] = re.findall("[0-9]{4}", re.findall("Né.*", li)[0])[0]   
        except:
            ""
        try:
            personne["ddm"] = re.findall("[0-9]{4}", re.findall("Décédé.*", li)[0])[0]
        except:
            ""
        try:
            ldn = re.findall("à [ '\w+]*", re.findall("Né.*", li)[0])[0].replace("à ", "")
        except:
            ""
        try:
            ldn = re.findall("- [ '\w+]*", re.findall("Né.*", li)[0])[0].replace("- ", "").replace("à ","")
        except:
            ""
        try:
            ldm = re.findall("à [ '\w+]*", re.findall("Décédé.*", li)[0])[0].replace("à ", "")
        except:
            ""
        try:
            ldm = re.findall("- [ '\w+]*", re.findall("Décédé.*", li)[0])[0].replace("- ", "").replace("à ","")
        except:
            ""

    personne["ldn"] = ldn
    personne["ldm"] = ldm
    
    # date_naissance = person_data_perso.find("li").text
    # print(date_naissance)
    # print(re.search("Né le .* -", date_naissance))
    
    
    # Récupération des parents
    try:
        parents_data = person_html.find("div", {"id":"parents"})
        personne["pere"] = parents_data.findAll("a")[0].text
        personne["mere"] = parents_data.findAll("a")[1].text
    except:
        print("Erreur parents")

    
    if(personne["genre"] == "H"):
        try:
            data = getEpouxEnfant(data, person_html)
        except:
            print("Err Epoux/Enfant")
    print(personne)
    data.append(personne)
    return data

    

In [125]:
test = """Né vers 1619 - à Souvert (commune de Chissey en Morvan ou Lucenay l'Evèque)"""

try:
    res = re.findall("à [ '\w+]*", test)[0].replace("à ", "")
except:
    ""

try:
    res = re.findall("- [ '\w+]*", test)[0].replace("- ", "").replace("à ","")
except:
    ""
print(res)

Souvert 


In [126]:
def get_genre(url):
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')
    person_html = soup.find("div", {"id":"perso"})

    # Récupération des données concernant la personne (genre, nom, prenom)
    person_title = person_html.find("div", {"id":"person-title"})
    person_title = person_title.find("h1")

    genre = person_title.find("img")["title"]

    return genre

In [127]:
def getAllUrls(urls, lasturl):
    print(lasturl)
    
    session = HTMLSession()
    r = session.get(lasturl)
    soup = BeautifulSoup(r.html.html, 'html.parser')
    # print(soup)
    
    # Récupération de l'html de l'ensemble des données d'une personne
    person_html = soup.find("div", {"id":"perso"})
    # print("soup" + str(person_html))

    print("debut epouxenfant")
    person_union = person_html.find("ul", {"class":"fiche_union"})
    
    
    try:
        print("personunion")
        person_epoux = person_union.findChildren("li" , recursive=False)
        # print(person_epoux)
        print("personepoux")
        
        for epoux in person_epoux:
            urls.append("https://gw.geneanet.org/" + epoux.find("a")["href"])
            children = epoux.find_all("li")
            print("*****************************")
            print("fils de : " + lasturl)
            print(children)
            print("*****************************")
            try:
                for child in children:
                    print(child)
                    url = "https://gw.geneanet.org/" + child.find("a")["href"]
                    urls.append(url)
                    print("-------------------------------------------------------------------")
                    print("url actu : " + url)
                    print(urls)
                    print("-------------------------------------------------------------------")
                    if(get_genre(url) == "H"):
                        urls = getAllUrls(urls, url)
            except:
                print("Err Children")
            print("fin epoux")
    except:
        print("Err Epoux")
    print()
    print()
    print()
    print()
    print()
    print()

    return urls

url = "https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=jean&n=martin&oc=17"
urls = getAllUrls([], url)
for u in urls:
    print(u)
print(len(urls))

https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=jean&n=martin&oc=17
debut epouxenfant
personunion
personepoux
: [<li style="vertical-align:middle;list-style-type:circle">
Marié <em>vers 1640, Chissey en Morvan ou Lucenay l'Evèque, </em>
 avec <a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=mangeotte&amp;n=allyot">Mangeotte Allyot</a> dont
<ul>
<li style="vertical-align:middle;list-style-type:square;">
<a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=dominique&amp;n=martin&amp;oc=5">Dominique Martin</a> <bdo dir="ltr">1641-1681</bdo></li>
<li style="vertical-align:middle;list-style-type:disc;">
<a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=dimanche&amp;n=martin">Dimanche Martin</a> <bdo dir="ltr">ca 1645-</bdo></li>
</ul>
</li>, <li style="vertical-align:middle;list-style-type:circle">
Marié <em>vers 1656</em>
 avec <a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=benoiste&amp;n=alliot">Benoiste Alliot</a> <bdo dir="ltr">ca 1634-

In [128]:
# Récupération du code source de chaque url correspondant à une personne
# {"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"}
urlDerAscendant = ""
# for url in allArbreUrl:
for i in range(1):
    url = "https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois"
    print("url de base : " + url)
    data = []
    urls = []
    
    urlDerAscendant = getUrlDerAscendant(url)
    print("url dernier ascendant : " + urlDerAscendant)
    # urls = getAllUrls(urls, url)
    data = getDataUrl(data, urlDerAscendant)
    print(data)
print(len(data))


url de base : https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois
Pas d'autre ascendent
url dernier ascendant : https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=mathieu&n=martin
https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=mathieu&n=martin
Erreur parents
debut epouxenfant
personunion
personepoux
[<li style="vertical-align:middle;list-style-type:disc;">
<a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=martin&amp;n=martin">Martin Martin</a> <bdo dir="ltr">ca 1618-</bdo></li>, <li style="vertical-align:middle;list-style-type:square;">
<a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=jean&amp;n=martin&amp;oc=17">jean Martin</a> <bdo dir="ltr">ca 1619-/1684</bdo></li>, <li style="vertical-align:middle;list-style-type:square;">
<a href="marmi1?lang=fr&amp;pz=mathieu&amp;nz=martin&amp;p=leonard&amp;n=martin&amp;oc=1">Léonard Martin</a> <bdo dir="ltr">?1622-</bdo></li>, <li style="vertical-align:middle;list-style-type:square;">
<a href="marm